# Exercise Classification Project

_Jorge Esteban Mendoza Ortiz_. Applied Mathematician at UNAM.

[esteban.mendoza@outlook.com](mailto:esteban.mendoza@outlook.com)

The so-called Human Activity Recognition (HAR) is a research area whose objective is to measure and understand human physical interactions with its environment with the use of sensors and observations of many kinds. For a wider view on HAR, you can refer to the corresponding [Wikipedia article](https://en.wikipedia.org/wiki/Activity_recognition).

HAR has attracted attention in recent years due to our increasing ability to produce large amounts of data with the use of devices such as _Jawbone Up_, _Nike FuelBand_, and _Fitbit_, that take advantage of sensors such as accelerometers to monitor the physical performance of an individual while performing exercises or other kinds of activities.

As one might expect, there are many potential applications for HAR, like: elderly monitoring, life log systems for monitoring energy expenditure and for supporting weight-loss programs, and digital assistants for weight lifting exercises.

The purpose of this project is to use the [_Weight Lifting Exercises (WLE)_ Dataset](http://groupware.les.inf.puc-rio.br/har#weight_lifting_exercises) to create a classifier that predicts whether an exercise is being properly performed or not. The _WLE_ Dataset was created from measurements of accelerometers placed on the belt, forearm, arm, and dumbbell of 6 participants. The participants were then asked to perform an exercise in 5 different ways - 4 of which constitute common mistakes. Our specific goal is to build a model that is capable of predicting the _classe_ variable in the dataset, that labels the manner in which the exercise was performed.

The present report will lay-out the process of building and implementing the model.

## Exploratory data analysis and pre-processing

According to the [Groupware@LES website](http://groupware.les.inf.puc-rio.br/har#weight_lifting_exercises) that hosts the original dataset:

> Six young health participants were asked to perform one set of 10 repetitions of the Unilateral Dumbbell Biceps Curl [[example video](https://youtu.be/YxtwA7XRK_g)] in five different fashions: exactly according to the specification (Class A), throwing the elbows to the front (Class B), lifting the dumbbell only halfway (Class C), lowering the dumbbell only halfway (Class D) and throwing the hips to the front (Class E).

> Class A corresponds to the specified execution of the exercise, while the other 4 classes correspond to common mistakes.

In [5]:
# Creating data directory
if (!dir.exists("data"))
    dir.create(file.path("data"))

# Downloading data
url_training = "https://d396qusza40orc.cloudfront.net/predmachlearn/pml-training.csv"
url_testing = "https://d396qusza40orc.cloudfront.net/predmachlearn/pml-testing.csv"

if (!file.exists("data/pml-training.csv"))
    download.file(url_training, destfile = "data/pml-training.csv")
if (!file.exists("data/pml-testing.csv"))
    download.file(url_testing, destfile = "data/pml-testing.csv")

In [6]:
# Loading data
training = read.csv("data/pml-training.csv", na.strings=c("NA",""))
testing = read.csv("data/pml-testing.csv", na.strings=c("NA",""))

Now, we will only point out to the few important characteristics and details found in the dataset that required some kind of pre-processing before we could fit the model.

First of all, we must note that from the 160 available variables in the dataset, we will only use 54 of them. The reasons are the following:

- We plan to fit a random forest model that allows us to predict the ```classe``` variable. In order to fit this model, we need our datasets to contain no NAs and we will also drop some unnecessary varilables.
- The first 7 variables in the dataset consist of a redundant variable with the index of each register, usernames and timestamps, that are unnecessary in the model.
- After some exploration we could determine that only 406 of the registers contained no NAs in all the variables, which constitutes little more than 2% of the whole data. This can be verified with the following commmand ```sum(apply(!is.na(training), 1, all))```.
- Furthermore, the NAs are concentrated along more than 100 variables, each of which have more than 95% of NAs on its registers.

Taking the above points into consideration we will proceed to preprocess our data in order to have a suitable form of it

In [7]:
# Dropping index, time stamps and usernames
training = training[,7:160]
testing = testing[,7:160]

## Dropping mostly NAs varilables
mostly_data = apply(!is.na(training), 2, mean) > 0.95

training = training[,mostly_data]
testing = testing[,mostly_data]

## Model training

We first will load up the necessary libraries to perform the model fitting. These are as follows.

In [ ]:
# Required libraries
library(caret)
library(randomForest)
library(doParallel)

In order to improve the fitting of the model, we will setup a parallel cluster. 

In [9]:
# Parallel processing
cluster <- makeCluster(detectCores())
registerDoParallel(cluster)

Before we proceed to train the model, we will generate a partition of the training dataset so we can estimate the out-of-sample error of the model.

In [10]:
# Creating validation set
set.seed(2412)
inTrain = createDataPartition(training$classe, p = 0.8, list = FALSE)

train0 = training[inTrain,]
validation = training[-inTrain,]

Now, we proceed to fit a random forest model with ```caret::train()``` with the following options:
- We select ```method = "rf"```, which calls the randomForest library, already attached.

- Within the call to ```caret::trainControl()```, we choose to perform cross-validation 5 times with the k-fold method, and

- We setup ```allowParallel = TRUE``` so it can perform a parallel computation with the ```doParallel``` backend activated.

In [11]:
h = train(classe ~ ., data=training, method = "rf",
          trControl = trainControl(method = "cv", number = 5, 
                                   allowParallel = TRUE))

Lastly, we shut down the cluster and register sequencing processing.

In [12]:
## Shutting down parallel
stopCluster(cluster)
registerDoSEQ()

## Model performance

The default output of the model object as outlined will provide us with quite some information about the performance of the model.

In [13]:
print(h)

Random Forest 

19622 samples
   53 predictor
    5 classes: 'A', 'B', 'C', 'D', 'E' 

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 15696, 15698, 15699, 15697, 15698 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
   2    0.9961780  0.9951655
  27    0.9983693  0.9979373
  53    0.9960251  0.9949721

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 27.


As it was already explained, the in-sample accuracy has been estimated using 5 k-folds cross validations, in such a way that the program sampled around 80% of the data, fitted the model and performed a validation on the 20% of the dataset held. The in-sample acuracy is estimated to be around 0.0.9983693, which constitutes a highly accurate fit.

In order to get an estimation of the out-of-sample error, we will compare the prediction ability of the model, using it in the random portion of the set that was left out before training it. This portion represent approximately the 20% of the data.

In [21]:
## Out-of-sample accuracy
output = predict(h, validation)
confusionMatrix(output, validation$classe)

Confusion Matrix and Statistics

          Reference
Prediction    A    B    C    D    E
         A 1116    0    0    0    0
         B    0  759    0    0    0
         C    0    0  684    0    0
         D    0    0    0  643    0
         E    0    0    0    0  721

Overall Statistics
                                     
               Accuracy : 1          
                 95% CI : (0.9991, 1)
    No Information Rate : 0.2845     
    P-Value [Acc > NIR] : < 2.2e-16  
                                     
                  Kappa : 1          
 Mcnemar's Test P-Value : NA         

Statistics by Class:

                     Class: A Class: B Class: C Class: D Class: E
Sensitivity            1.0000   1.0000   1.0000   1.0000   1.0000
Specificity            1.0000   1.0000   1.0000   1.0000   1.0000
Pos Pred Value         1.0000   1.0000   1.0000   1.0000   1.0000
Neg Pred Value         1.0000   1.0000   1.0000   1.0000   1.0000
Prevalence             0.2845   0.1935   0.1744   0.16

We can appreciate that we get 100% accuracy even over a considerable portion of the dataset of almost 4000 registers. That being said, we will stick to the more conservative 95% confidence interval of (0.9991, 1) for estimating the out-of-sample error.